<a href="https://colab.research.google.com/github/Hirak-Pandya/Stockmarketprediction/blob/main/IBM_Summer_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Portfolio optimization using Efficient Frontier 

NSE Equity List - https://docs.google.com/spreadsheets/d/1Du-_917AzZQr7txoJFQvrXzEjAAfLSsnJaCiNSGnYzI/edit?usp=sharing

In [2]:
from pandas_datareader import data as web
import pandas as pd
import numpy as np
import seaborn as sns
import math
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

In [3]:
capital=int(input("Enter capital you are going to invest : "))

Enter capital you are going to invest : 100000


In [4]:
nstocks = int(input("Enter the no. of stocks :"))
assets = [ ]
for i in range(0,nstocks):
  
  data = input("Enter stock name :")
  data+='.ns' 
  assets.append(data)
  i+=1
print(assets)

Enter the no. of stocks :2
Enter stock name :REL
Enter stock name :COALINDIA
['REL.ns', 'COALINDIA.ns']


In [5]:
StartDate = '2020-01-01'
#EndDate = datetime.today().strftime('%Y-%m-%d')
EndDate = "2021-07-09"


In [6]:
df = pd.DataFrame()
for stocks in assets:
  df[stocks] = web.DataReader(stocks,data_source='yahoo',start= StartDate, end = EndDate)['Adj Close']
print(df)

RemoteDataError: ignored

In [ ]:
stocks_pct_change = df.pct_change().dropna()
stocks_pct_change.head()

In [ ]:
variance_stocks = stocks_pct_change.var()
variance_stocks


In [ ]:
std_stocks = np.std(stocks_pct_change)*np.sqrt(250).mean()
print(std_stocks)

In [ ]:
std_stocks.plot(kind='bar')

In [ ]:
cov_matrix = stocks_pct_change.cov()
cov_matrix

In [ ]:
stocks_corr = stocks_pct_change.corr()
sns.heatmap(stocks_corr, 
        xticklabels=stocks_corr.columns,
        yticklabels=stocks_corr.columns,
            cmap="Greens")

In [ ]:
yearly_return = df.resample('Y').first().pct_change().dropna()
annual_return = yearly_return.mean()
annual_return


In [ ]:
assets1 = pd.concat([annual_return, std_stocks], axis=1)
assets1.columns = ['Returns', 'Volatility']
assets1

In [ ]:
p_ret = [] # Define an empty array for portfolio returns
p_vol = [] # Define an empty array for portfolio volatility
p_weights = [] # Define an empty array for asset weights

num_assets = len(df.columns)
num_portfolios = 10000

for portfolio in range(num_portfolios):
    weights = np.random.random(num_assets)
    weights = weights/np.sum(weights)
    p_weights.append(weights)
    returns = np.dot(weights, annual_return) # Returns are the product of individual expected returns of asset and its 
                                      # weights 
    p_ret.append(returns)
    var = cov_matrix.mul(weights, axis=0).mul(weights, axis=1).sum().sum()# Portfolio Variance
    sd = np.sqrt(var) # Daily standard deviation
    ann_sd = sd*np.sqrt(250) # Annual standard deviation = volatility
    p_vol.append(ann_sd)

In [ ]:
data = {'Returns':p_ret, 'Volatility':p_vol}

for counter, symbol in enumerate(df.columns.tolist()):
    #print(counter, symbol)
    data[symbol+' weight'] = [w[counter] for w in p_weights]

In [ ]:
portfolios  = pd.DataFrame(data) 
portfolios # Dataframe of the 10000 portfolios created

In [ ]:
portfolios.plot.scatter(x='Volatility', y='Returns', marker='o', alpha=0.3, grid=True, figsize=[10,10])

In [ ]:
min_vol_port = portfolios.iloc[portfolios['Volatility'].idxmin()]
# idxmin() gives us the minimum value in the column specified.                               
min_vol_port

In [ ]:
plt.subplots(figsize=[10,10])
plt.scatter(portfolios['Volatility'],portfolios['Returns'],marker='o', s=10, alpha=0.3)
plt.scatter(min_vol_port[1],min_vol_port[0],marker='*',s=500 ,color ='r')

In [ ]:
# Finding the optimal portfolio
rf = 0.01 #risk factor
optimal_risk_port = portfolios.iloc[((portfolios['Returns']-rf)/portfolios['Volatility']).idxmax()]
optimal_risk_port


In [ ]:
optimal_risk_port*capital

In [ ]:
portfolios_new = portfolios.reset_index(drop=False)
row_no = portfolios_new[['index']].iloc[((portfolios['Returns']-rf)/portfolios['Volatility']).idxmax()]
row_no


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# fix_yahoo_finance is used to fetch data 
import fix_yahoo_finance as yf
yf.pdr_override()

In [ ]:
start = '2020-01-01'
end = datetime.today().strftime('%Y-%m-%d')

symbol='reliance.ns'
# Read data 
df = yf.download(symbol,start,end)

# View Columns
df.head()

In [ ]:
n = 200 # number of periods
df['MA_High'] = df['High'].rolling(n).mean()
df['MA_Low'] = df['Low'].rolling(n).mean()

In [ ]:
fig = plt.figure(figsize=(14,10))
ax1 = plt.subplot(2, 1, 1)
ax1.plot(df['Adj Close'])
ax1.plot(df['MA_High'], label='Moving Average of High')
ax1.plot(df['MA_Low'], label='Moving Average of Low')
ax1.set_title('Stock '+ symbol +' Closing Price')
ax1.set_ylabel('Price')
ax1.legend(loc='best')

ax2 = plt.subplot(2, 1, 2)
ax2.plot(df['MA_High'], label='Moving Average of High')
ax2.plot(df['MA_Low'], label='Moving Average of Low')
ax2.grid()
ax2.legend(loc='best')
ax2.set_ylabel('Moving Average of High and Low')
ax2.set_xlabel('Date')